In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from textblob import TextBlob
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [ ]:
df1 = pd.read_csv(r'/content/gdrive/MyDrive/project-ml/Input.csv', encoding="utf-8")
df2 = pd.DataFrame(df1)
df2

,URL_ID,URL
0,37,https://insights.blackcoffer.com/ai-in-healthc...
1,38,https://insights.blackcoffer.com/what-if-the-c...
2,39,https://insights.blackcoffer.com/what-jobs-wil...
3,40,https://insights.blackcoffer.com/will-machine-...
4,41,https://insights.blackcoffer.com/will-ai-repla...
...,...,...
109,146,https://insights.blackcoffer.com/blockchain-fo...
110,147,https://insights.blackcoffer.com/the-future-of...
111,148,https://insights.blackcoffer.com/big-data-anal...
112,149,https://insights.blackcoffer.com/business-anal...


In [ ]:
#Data Exctraction
for i in df2.index:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    }
    page = requests.get(df2['URL'][i], headers=headers)
    soup = BeautifulSoup(page.content, "html.parser")
    text=soup.get_text().encode('cp1252', errors='ignore')
    file = open('/content/gdrive/MyDrive/project-ml/articles/'+str(df2['URL_ID'][i])+'.txt','w')
    file.write(str(text))
    file.close()


In [ ]:
#Maters and stopwords
lines_list = []
for line in text:
  line = str(line)
  stripped_line = line.strip()
  lines_list.append(stripped_line)
with open(r"/content/gdrive/MyDrive/project-ml/StopWords/StopWords_Currencies.txt",'rb+') as file:
  for line in enumerate(file):
    line = str(line)
    stripped_line = line.strip()
    lines_list.append(stripped_line)
file = open(r"/content/gdrive/MyDrive/project-ml/StopWords/StopWords_DatesandNumbers.txt", 'rb+')
lines_list = []
for line in file:
  line = str(line)
  stripped_line = line.strip()
  lines_list.append(stripped_line)
file = open(r"/content/gdrive/MyDrive/project-ml/StopWords/StopWords_Generic.txt",'rb+')
for line in file:
  line = str(line)
  stripped_line = line.strip()
  lines_list.append(stripped_line)
file = open("/content/gdrive/MyDrive/project-ml/StopWords/StopWords_GenericLong.txt", "rb+")
for line in file:
  line = str(line)
  stripped_line = line.strip()
  lines_list.append(stripped_line)
file = open("/content/gdrive/MyDrive/project-ml/StopWords/StopWords_Geographic.txt", "rb+")
for line in file:
  line = str(line)
  stripped_line = line.strip()
  lines_list.append(stripped_line)
file = open("/content/gdrive/MyDrive/project-ml/StopWords/StopWords_Names.txt", "rb+")
for line in file:
  line = str(line)
  stripped_line = line.strip()
  lines_list.append(stripped_line)
new_stopwords = lines_list
positive_wds = []
file = open("/content/gdrive/MyDrive/project-ml/positive-words.txt", "rb+")
for line in file:
  line = str(line)
  positive_wds = file.read().split()
negative_wds = []
file = open("/content/gdrive/MyDrive/project-ml/negative-words.txt", "rb+")
for line in file:
  line = str(line)
  negative_wds = file.read().split()
stpwrd = nltk.corpus.stopwords.words('english')
stpwrd.extend(new_stopwords)
nltk.download('punkt')
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
9#Data Analysis
def syllable_count(word):
    count = 0
    vowels = "aeiouy"
    if word[0] in vowels:
        count += 1
    for index in range(1, len(word)):
        if word[index] in vowels and word[index - 1] not in vowels:
            count += 1
            if word[index].endswith("e"):
                count -= 1
    if count == 0:
        count += 1
    return count

URL_ID = []
URL = []
POSITIVE_SCORE = []
NEGATIVE_SCORE = []
POLARITY_SCORE = []
SUBJECTIVITY_SCORE = []
AVG_SENTENCE_LENGTH = []
PERCENTAGE_OF_COMPLEX_WORDS = []
FOG_INDEX = []
AVG_NUMBER_OF_WORDS_PER_SENTENCE = []
COMPLEX_WORD_COUNT = []
WORD_COUNT = []

for i in df2.index:
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.12; rv:55.0) Gecko/20100101 Firefox/55.0',
    }
    page = requests.get(df2['URL'][i], headers=headers)
    soup = BeautifulSoup(page.content, "lxml")
    text=soup.get_text().encode('cp1252', errors='ignore')
    text_tokens = word_tokenize(str(text))
    filtered_sentence = [w for w in text_tokens if not w.lower() in new_stopwords]
    
    total = len(filtered_sentence)
    pos = len([i for i in filtered_sentence if i in positive_wds])
    neg = len([i for i in filtered_sentence if i in negative_wds])
    Polarity_Score = (pos - neg)/((pos + neg) + 0.000001)
    Subjectivity_Score = (pos + neg)/ ((total) + 0.000001)
    words = len(str(text).split())
    sentence = len(text_tokens)
    Average_Sentence_Length = words/sentence
    Complex_Word = syllable_count(filtered_sentence)
    Percentage_of_Complex_words = Complex_Word / words
    Fog_Index = 0.4 * (Average_Sentence_Length + Percentage_of_Complex_words)
    Average_Number_of_Words = words / sentence
    
    URL_ID.append(df2['URL_ID'][i])
    
    URL.append(df2['URL'][i])
    
    POSITIVE_SCORE.append(pos)
    
    NEGATIVE_SCORE.append(neg)
    
    POLARITY_SCORE.append(Polarity_Score)
    
    SUBJECTIVITY_SCORE.append(Subjectivity_Score)
    
    AVG_SENTENCE_LENGTH.append(Average_Sentence_Length)
    
    PERCENTAGE_OF_COMPLEX_WORDS.append(Percentage_of_Complex_words)
    
    FOG_INDEX.append(Fog_Index)
   
    AVG_NUMBER_OF_WORDS_PER_SENTENCE.append(Average_Number_of_Words)
    
    COMPLEX_WORD_COUNT.append(Complex_Word)
    
    WORD_COUNT.append(words)

In [ ]:
#Output
df3 = pd.DataFrame({'URL_ID' : URL_ID,
                    'URL' : URL,
                    'POSITIVE SCORE' : POSITIVE_SCORE,
                    'NEGATIVE SCORE' : NEGATIVE_SCORE,
                    'POLARITY SCORE' : POLARITY_SCORE,
                    'SUBJECTIVITY SCORE' : SUBJECTIVITY_SCORE,
                    'AVG SENTENCE LENGTH' : AVG_SENTENCE_LENGTH,
                    'PERCENTAGE OF COMPLEX WORDS' : PERCENTAGE_OF_COMPLEX_WORDS,
                    'FOG INDEX' : FOG_INDEX,
                    'AVG NUMBER OF WORDS PER SENTENCE' : AVG_NUMBER_OF_WORDS_PER_SENTENCE,
                    'COMPLEX WORD COUNT' : COMPLEX_WORD_COUNT,
                    'WORD COUNT' : WORD_COUNT,}, 
                    columns=['URL_ID','URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
                             'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS',
                             'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT',
                             'WORD COUNT'])
df4 = pd.DataFrame(df3)
df4.to_csv(r'Output_Data.csv', index=False, header=True)

In [ ]:
import shutil

src_path = r"/content/Output_Data.csv"
dst_path = r"/content/gdrive/MyDrive/project-ml/Output_Data.csv"
shutil.move(src_path, dst_path)

'/content/gdrive/MyDrive/project-ml/Output_Data.csv'